# Importing Training and Testing Data

In [1]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('data/train.csv')

X_t = train_data.drop(['category', 'ID'], axis=1)
y_t = train_data['category']

test_data = pd.read_csv('data/test.csv')

# Approach 1: Using `sklearn`

## Training Data without any preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, test_size=0.2, random_state=42)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

print(rf.score(X_test, y_test))


## Removing outliers using Isolation Forest and training again

In [ ]:
from pipeline import OutlierDetection
# import importlib
# importlib.reload(OutlierDetection)

od = OutlierDetection('isolation_forest')
X_t_iso, y_t_iso = od.transform(X_t, y_t)

X_train, X_test, y_train, y_test = train_test_split(X_t_iso, y_t_iso, test_size=0.2, random_state=42)
rf.fit(X_train, y_train)
print("Accuracy:", rf.score(X_test, y_test))


## Removing outliers using LOF and training again

In [ ]:
from pipeline import OutlierDetection

lof = OutlierDetection('lof')
X_t_lof, y_t_lof = lof.transform(X_t, y_t)

X_train, X_test, y_train, y_test = train_test_split(X_t_lof, y_t_lof, test_size=0.2, random_state=42)
rf.fit(X_train, y_train)
print("Accuracy:", rf.score(X_test, y_test))

## Now also doing dimensionality reduction using LDA

In [ ]:
from pipeline import DimReduction

dr = DimReduction('pca', 19)
X_t_pca, y_t_pca = dr.transform(X_t, y_t)


clf = IsolationForest().fit(X_t_lda)
y_pred = clf.predict(X_t_lda)

X_t_lda_iso = X_t_lda[y_pred == 1]
y_t_lda_iso = y_t[y_pred == 1]

X_train, X_test, y_train, y_test = train_test_split(X_t_lda_iso, y_t_lda_iso, test_size=0.2, random_state=0)

rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
print(X_t.shape)

## Generating submission file

In [ ]:
from datetime import datetime

X_test = test_data.drop(['ID'], axis=1)

X_test_lda = lda.transform(X_test)

rf.fit(X_t_lda_iso, y_t_lda_iso)
y_pred = rf.predict(X_test_lda)

submission = pd.DataFrame({'ID': test_data['ID'], 'Category': y_pred})
submission.to_csv(f"submissions/submission_{(datetime.now()).strftime('%Y_%m_%d-%H_%M')}.csv", index=False)

# Approach 2: Pipelining the process

In [2]:
import metaclass as mc
import pipeline_components as pc
import importlib
importlib.reload(mc)

<module 'metaclass' from '/Users/Atharv/Documents/uni/Semester 4/CSE342 - Statistical Machine Learning [SML]/Project/SML-Project/metaclass.py'>

In [3]:
pipeline = mc.Pipeline(
    clustering_alg=None,
    dim_reduction_alg_1="pca",
    dim_reduction_alg_2="lda",
    outlier_detection_alg="lof",
    classification_alg="logistic",
    ensemble_alg=None
)
pipeline.fit(X_t, y_t)
print("Pipeline done")
# print("Pipeline score: ", pipeline.score(X_t, y_t))
cv_scores = pipeline.cross_validate(X_t, y_t)

Standardizing data...
Currently at clustering: None
Currently at dim reduction: pca
Currently at dim reduction: lda
Currently at outlier removal: lof
Outlier percentage: 0.017269736842105265
Currently at classifier: logistic
Currently at endsembling: None
Done!
Pipeline done


/Users/Atharv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Atharv/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [6]:
# cv_scores.sort()
cv_scores

(0.7895230385212171, 0.034668283373380064)

In [ ]:
pipeline.generate_submission(test_data)
